In [1]:
# INTRODUCTION:

# Automated Script to export all the binary data: input, weights, biases and intermediate output 
# for three example inputs:  x_val[0], x_val[1] and x_val[2] in different folders.

# WHAT IT DOES:

# Once executed this script will create three folders: Test_Input0, Test_Input1 and Test_Input2
# with all the required binary files in the current directory where you execute the script.

# PREREQUISITE:

# Please note that for this script to execute the model: CNN_TinyImageNet_2.h5 
# must be present in the same folder from where this script is executed.
# And data must be present at: C://Temp//CPRE482X//Lab1//Files//Dataset//tiny-imagenet-200

In [2]:
# Install necessary libraries
import pathlib
import numpy as np
import tensorflow as tf
# Please make sure GPU is disabled for this inference part of the lab
# Disable GPU and enable xla flags
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model

2021-08-25 20:40:50.301032: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
# FIRST LOAD THE DATA
# Make sure that unzipped tiny-imagenet-200 folder is placed in the current directory
#define path to the dataset
# Put the correct path to the dataset
path = pathlib.Path('/local/tiny-imagenet-200')

In [4]:
# Map the wnids to integer labels to words
with open(os.path.join(path, 'wnids.txt'), 'r') as f:
  wnids = [x.strip() for x in f]


# Map wnids to integer labels
wnid_to_label = {wnid: i for i, wnid in enumerate(wnids)}
label_to_wnid = {v: k for k, v in wnid_to_label.items()}

# Use words.txt to get names for each class
with open(os.path.join(path, 'words.txt'), 'r') as f:
  wnid_to_words = dict(line.split('\t') for line in f)
  for wnid, words in wnid_to_words.items():
      wnid_to_words[wnid] = [w.strip() for w in words.split(',')]
class_names = [wnid_to_words[wnid] for wnid in wnids]

In [5]:
# Import necessary packages
from __future__ import print_function
from builtins import range
from six.moves import cPickle as pickle
import numpy as np
import os
import platform
from tqdm import tqdm
import random
from imageio import imread
from imageio import imsave


# Function for loading the tiny imagenet data
def load_tiny_imagenet(path, is_training=True, dtype=np.float32, subtract_mean=True, debug=False, debug_nclass=3):
    """
    Load TinyImageNet. Each of TinyImageNet-100-A, TinyImageNet-100-B, and
    TinyImageNet-200 have the same directory structure, so this can be used
    to load any of them.

    Note: The original implementation loaded data as NCHW, I (tyler) changed it to NHWC

    Inputs:
    - path: String giving path to the directory to load.
    - is_training: If True, dont load testing data, if False, dont load training and val data
        Note: Must always load training data in order to subtract_mean.
    - dtype: numpy datatype used to load the data.
    - subtract_mean: Whether to subtract the mean training image.
    - debug: Whether or not to load a small number of classes for debugging

    Returns: A dictionary with the following entries:
    - class_names: A list where class_names[i] is a list of strings giving the
      WordNet names for class i in the loaded dataset.
    - X_train: (N_tr, 64, 64, 3) array of training images
    - y_train: (N_tr,) array of training labels
    - X_val: (N_val, 64, 64, 3) array of validation images
    - y_val: (N_val,) array of validation labels
    - X_test: (N_test, 64, 64, 3) array of testing images.
    - y_test: (N_test,) array of test labels; if test labels are not available
      (such as in student code) then y_test will be None.
    - mean_image: (64, 64, 3) array giving mean training image
    - label_to_wnid: dictionary with mapping from integer class label to wnid
    """
    # First load wnids
    with open(os.path.join(path, 'wnids.txt'), 'r') as f:
        wnids = [x.strip() for x in f]

    # Map wnids to integer labels
    wnid_to_label = {wnid: i for i, wnid in enumerate(wnids)}
    label_to_wnid = {v: k for k, v in wnid_to_label.items()}

    # Use words.txt to get names for each class
    with open(os.path.join(path, 'words.txt'), 'r') as f:
        wnid_to_words = dict(line.split('\t') for line in f)
        for wnid, words in wnid_to_words.items():
            wnid_to_words[wnid] = [w.strip() for w in words.split(',')]
    class_names = [wnid_to_words[wnid] for wnid in wnids]

    if debug:
        print('Debug is on! Only loading %d / %d training classes.'
                  % (debug_nclass, len(wnids)))

    # Next load training data.
    X_train, y_train = [], []
    train_wnids = wnids[:debug_nclass] if debug else wnids
    for i, wnid in tqdm(enumerate(train_wnids), total=len(train_wnids)):
        # To figure out the filenames we need to open the boxes file
        boxes_file = os.path.join(path, 'train', wnid, '%s_boxes.txt' % wnid)
        with open(boxes_file, 'r') as f:
            filenames = [x.split('\t')[0] for x in f]
        num_images = len(filenames)

        X_train_block = np.zeros((num_images, 64, 64, 3), dtype=dtype)
        y_train_block = wnid_to_label[wnid] * \
                        np.ones(num_images, dtype=np.int64)
        for j, img_file in enumerate(filenames):
            img_file = os.path.join(path, 'train', wnid, 'images', img_file)
            img = imread(img_file)
            if img.ndim == 2:   ## grayscale file
                img.shape = (64, 64, 1)
            X_train_block[j] = img
        X_train.append(X_train_block)
        y_train.append(y_train_block)

    # We need to concatenate all training data
    X_train = np.concatenate(X_train, axis=0)
    y_train = np.concatenate(y_train, axis=0)

    # Next load validation data
    X_val, y_val = None, None
    if is_training:
        print('loading validation data')
        with open(os.path.join(path, 'val', 'val_annotations.txt'), 'r') as f:
            img_files = []
            val_wnids = []
            for line in f:
                img_file, wnid = line.split('\t')[:2]
                img_files.append(img_file)
                val_wnids.append(wnid)
            num_val = len(img_files)
            y_val = np.array([wnid_to_label[wnid] for wnid in val_wnids])
            X_val = np.zeros((num_val, 64, 64, 3), dtype=dtype)
            for i, img_file in tqdm(enumerate(img_files), total=len(img_files)):
                img_file = os.path.join(path, 'val', 'images', img_file)
                img = imread(img_file)
                if img.ndim == 2:
                    img.shape = (64, 64, 1)
                X_val[i] = img

    mean_image = None
    if subtract_mean:
        mean_image = X_train.mean(axis=0)
        if is_training:
            X_train -= mean_image[None]
            X_val -= mean_image[None]
        else:
            X_test -= mean_image[None]

    if not is_training:
        X_train = None
        y_train = None

    return {
      'class_names': class_names,
      'X_train': X_train,
      'y_train': y_train,
      'X_val': X_val,
      'y_val': y_val,
      'mean_image': mean_image,
      'label_to_wnid': label_to_wnid
    }


In [6]:
print ("Loading Tiny-Imagenet Dataset for training and validation data")
dataset_train_val = load_tiny_imagenet(path, is_training = True, dtype=np.float32, subtract_mean=False)
x_val = dataset_train_val["X_val"] # 10000 images when each is a 64*64*3
y_val = dataset_train_val["y_val"] # 10000 elements

Loading Tiny-Imagenet Dataset for training and validation data


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:59<00:00,  3.37it/s]


loading validation data


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:06<00:00, 1616.57it/s]


In [7]:
# Normalize all the float data between 0 and 1
x_val = x_val/255.0

In [8]:
# one hot encode y data
from tensorflow.keras.utils import to_categorical
num_classes = 200
y_val = to_categorical(y_val, num_classes)

In [9]:
# To do:  Load the H5 model and print summary. Please make sure the h5 model file is present in the current directory.
# Sample code to load the model
model = tf.keras.models.load_model('/local/jupyter/cpre482x-lab1/Inference/Template_Jupyter/CNN_tinyimagenet.h5')

2021-08-25 20:42:04.263002: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-08-25 20:42:04.286818: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-08-25 20:42:04.290940: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: cpre482-03.ece.iastate.edu
2021-08-25 20:42:04.290956: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: cpre482-03.ece.iastate.edu
2021-08-25 20:42:04.294916: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.63.1
2021-08-25 20:42:04.294974: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.63.1
2021-08-25 20:42:04.294983: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.63.1
2021-08-25 20:42:04.306627: I 

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 60, 32)        2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 32)        25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 64)        3

In [11]:
# Create three folders: one for each of the three inputs. Each folder will contain the exported inputs, weights, biases, intermediate input binary files
import os

# Create folder only if they have not been created in the past

# Get the absolute path to the current directory
path_dir = os.path.abspath(os.getcwd())

# Create folders if not already created
if(os.path.isdir("Test_Input0") and os.path.isdir("Test_Input1") and os.path.isdir("Test_Input2")):
    print("Required folders already created and so skipping...")
else:
    os.mkdir("Test_Input0")
    os.mkdir("Test_Input1")
    os.mkdir("Test_Input2")
    print("Created Required folders...")

    


Required folders already created and so skipping...


In [12]:
# Save the test input images as binary files to the corresponding folders
for i in range(3):
    input_image = np.asarray(x_val[i].flatten())
    f =open(os.path.abspath(os.getcwd()) + "/Test_Input"+str(i)+"/"+"input.bin","wb")
    input_image.tofile(f)
    f.close()
    
print("All the test images successfully exported to each input folder...")    
    


All the test images successfully exported to each input folder...


In [13]:
# Export the weights and biases to all the three folders
# Save the weights and biases of all the convolution and dense layer (pooling layers has no weights)
# For conv layer we save weights and biases as: convi_weights.bin, convi_biases.bin ... i.e. i=1,2,3 ...
import re # for pattern matching

for j in range(3):
    # Saving for the convolution layers
    conv_index=1 # layer index starts from one
    for i in range(len(model.layers)):

        if re.match("conv", model.layers[i].name):
            layer_weights = np.asarray(model.layers[i].get_weights()) # Get all weights including biases
            layer_weights_flat=np.asarray(layer_weights[0].flatten()) # Get weights
            layer_biases_flat=np.asarray(layer_weights[1].flatten()) # Get biases
            
            # Save the weights as binary files in the folder of all three inputs
            f =open(os.path.abspath(os.getcwd()) + "\\Test_Input"+str(j)+"\\"+"conv"+str(conv_index)+"_weights.bin","wb")
            layer_weights_flat.tofile(f)
            f.close()
            
             # Save the biases as binary files in the folder of all three inputs
            f =open(os.path.abspath(os.getcwd()) + "\\Test_Input"+str(j)+"\\"+"conv"+str(conv_index)+"_biases.bin","wb")
            layer_biases_flat.tofile(f)
            f.close()
            
            conv_index=conv_index+1 # increment index
            
    # Saving for the dense layers       
    dense_index=1
    for i in range(len(model.layers)):

        if re.match("dense", model.layers[i].name):
            layer_weights = np.asarray(model.layers[i].get_weights()) # Get all weights including biases
            layer_weights_flat=np.asarray(layer_weights[0].flatten()) # Get weights
            layer_biases_flat=np.asarray(layer_weights[1].flatten()) # Get biases
            
            # Save the weights as binary files in the folder of all three inputs
            f =open(os.path.abspath(os.getcwd()) + "/Test_Input"+str(j)+"/"+"dense"+str(dense_index)+"_weights.bin","wb")
            layer_weights_flat.tofile(f)
            f.close()
            
             # Save the biases as binary files in the folder of all three inputs
            f =open(os.path.abspath(os.getcwd()) + "/Test_Input"+str(j)+"/"+"dense"+str(dense_index)+"_biases.bin","wb")
            layer_biases_flat.tofile(f)
            f.close()
            
            dense_index=dense_index+1 # increment index
            
            
            
print("All the convolution and dense(fully connected) weights and biases successfully exported to each input folder")



All the convolution and dense(fully connected) weights and biases successfully exported to each input folder


/usr/local/anaconda3/envs/cpre482x_tf_gpu/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [14]:
# Export the intermediate layer outputs for each of the input for all of the layers

for j in range(3):
    
    for i in range(len(model.layers)):
        aux_model = tf.keras.Model(inputs=model.inputs,
                           outputs=[model.layers[i].output])
        
        # Store the intermediate output
        intermediate_layer_output = aux_model.predict(x_val[j].reshape(1,64,64,3))
        intermediate_layer_output_flat = np.asarray(intermediate_layer_output.flatten())

        # Save the Output in a binary file i.e. numpy.ndarray.to_file()
        f_output=open(os.path.abspath(os.getcwd()) + "/Test_Input"+str(j)+"/"+"layer_"+str(i)+"_output.bin","wb")
        intermediate_layer_output_flat.tofile(f_output)
        f_output.close()
        
print("All the corresponding intermediate layer outputs successfully exported to each input folder")


2021-08-25 20:42:04.756468: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


All the corresponding intermediate layer outputs successfully exported to each input folder
